In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.chdir('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys
sys.path.append("Marigold")

import torch
import numpy as np
import jhutil; jhutil.color_log(1111, )

 1111 


In [2]:
import sys
sys.argv = [
    "ludvig_uplift.py", 
    "--colmap_dir", "./dataset/llff_data/fern/", 
    "--gs_source", "./dataset/llff_data/fern/gs/point_cloud/iteration_30000/point_cloud.ply", 
    "--config", "configs/dif_NVOS.yaml", 
    "--height", "1199", 
    "--width", "1600", 
    "--tag", "fern"
]

In [ ]:
from ludvig_uplift import *

args = parse_args()
reproducibility(0)
model = LUDVIGUplift(args)

Reading camera 20/20
735471 Gaussians.


# 1. Uplift

In [5]:
t0 = time()
print("Uplifting features...")
directory = model.config['feature'].pop(
    'directory',
    os.path.join(model.colmap_dir, 'images')
)
dataset = config_to_instance(
    directory=directory,
    gaussian=model.gaussian,
    cameras=model.colmap_cameras,
    render_fn=model.render,
    scene=model.scene,
    height=model.img_height,
    width=model.img_width,
    **model.config.feature,
)
loader = iter(dataset)

Uplifting features...
Computing PCA...
   cccc  "cache file found, skipping apply_pca"
   aaaa  "cache file corrupted, executing apply_pca"


In [ ]:
features, _ = uplifting(
    loader,
    model.gaussian,
    prune_gaussians=model.config.get("prune_gaussians", None),
)
if model.config.get('normalize', False):
    print("l2-normalizing uplifted features.")
    features /= features.norm(dim=1, keepdim=True) + 1e-6
print(
    f"Total time for preprocessing + uplifting {len(model.colmap_cameras)} images: {round(time()-t0)}s"
)
model.features = features

20it [00:01, 10.20it/s]

Time for uplifting 20 views: 2.0s
Total time for preprocessing + uplifting 20 images: 76s
